In [73]:
import cv2
import numpy as np
import colorsys

In [74]:
bgr = cv2.imread('../data/HazeRemoval/3.jpg',1)
gsc = cv2.imread('../data/HazeRemoval/3.jpg',0)
hsv = cv2.cvtColor(bgr, cv2.COLOR_BGR2HSV)

In [75]:
cv2.imshow('image',bgr)
cv2.waitKey(10000)
cv2.destroyAllWindows()

In [76]:
s = hsv[:,:,1]
v = hsv[:,:,2]
sigma = 0.05
mean = 0
r = 15
beta = 1
sigmaMat = np.random.normal(mean,sigma,(hsv.shape[0],hsv.shape[1]))
#output = 0.121779 + 0.959710*v  - 0.780245*s + sigmaMat;
output = 0.1893 + 1.0267*v  - 1.2966*s + sigmaMat 

# Scaling output in 0 to 255
original_output = output.copy()
scale = output.max() - output.min()
output = output - output.min()
output = output/output.max()
output = output * 255
output = np.uint8(output)

# temp = np.zeros(output.shape)
# for i in range(0,temp.shape[0]):
#     for j in range(0,temp.shape[1]):
#         min_x = max(0,i-(r//2))
#         max_x = min(temp.shape[0]-1,i+(r//2))
#         min_y = max(0,j-(r//2))
#         max_y = min(temp.shape[1]-1,j+(r//2))
#         temp[i][j] = (output[min_x:max_x+1,min_y:max_y+1]).min()
# temp = np.uint8(temp)

kernel = np.ones((r,r),np.uint8)
depthMap = cv2.erode(output,kernel,iterations = 1)

In [77]:
cv2.imshow("Original Output",depthMap)
cv2.waitKey(100000)
cv2.destroyAllWindows()

In [78]:
r_for_guiding = r*r
redefinedDepthMap = np.uint8(cv2.ximgproc.guidedFilter(bgr,depthMap,r_for_guiding,0.001))

In [79]:
cv2.imshow("Refined Depth Map",redefinedDepthMap)
cv2.waitKey(100000)
cv2.destroyAllWindows()

In [80]:
tR = np.exp(-beta*redefinedDepthMap/255)

In [81]:
#Changing tr to stay in range 0.1 to 0.9
tr = np.zeros(tR.shape)
for i in range(0,tR.shape[0]):
    for j in range(0,tR.shape[1]):
        tr[i][j] = min(max(0.1,tR[i][j]),0.9)

In [82]:
#Estimating A using refined depth map
#Intensity of pixel with high depth value and high intensity can be estimated as atmospheric light
# depth_and_intensity = np.zeros((redefinedDepthMap.shape[0],redefinedDepthMap.shape[1],2))
# for i in range(0,depth_and_intensity.shape[0]):
#     for j in range(0,depth_and_intensity.shape[1]):
#         depth_and_intensity[i][j] = [redefinedDepthMap[i][j],hsv[i][j][2]]

#Candidates are top 0.1% pixels with max depth
#a = hsv[hsv[:,:,2] == hsv[:,:,2].max()][0]
A = [255,255,255]

In [83]:
I = np.zeros((tr.shape[0],tr.shape[1],3))
for i in range(0,tr.shape[0]):
    for j in range(0,tr.shape[1]):
        temp = bgr[i][j] - A
        temp = ((temp)/tr[i][j])
        #print(np.uint8(temp))
        I[i][j] = temp + A
I = np.uint8(abs(I))

In [84]:
cv2.imshow("Refined Depth Map",I)
cv2.waitKey(100000)
cv2.destroyAllWindows()